In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import json
import random
from http import HTTPStatus
import dashscope
from dashscope.api_entities.dashscope_response import Role

In [2]:
class AQuADataset(Dataset):
    def __init__(self, data_path, max_records=10):
        with open(data_path, 'r') as file:
            data = [json.loads(line) for line in file]
        data = data[:max_records]
        df = pd.DataFrame(data)
        df = df.head(max_records)
        
        def combine(row):
            choices = [f"{chr(65 + i)}: {option}" for i, option in enumerate(row['options'])]
            return f"Question: {row['question']} Choices: {', '.join(choices)}"
        
        self.inputs = df.apply(combine, axis=1).tolist()
        self.labels = [row['correct'] for row in data]
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

# 修改数据路径
aqua_data_path = '/data_zju_2/chenkerui/train.json'

# 实例化AQuA数据集
aqua_dataset = AQuADataset(aqua_data_path)

# 创建DataLoader
aqua_dataloader = DataLoader(aqua_dataset, batch_size=10, shuffle=False)

In [3]:
for batch in aqua_dataloader:
    inputs, labels = batch
    print(f"Inputs: {inputs}")
    print(f"Labels: {labels}")
    break

# 以下代码保持不变
dashscope.api_key = "sk-ea4a0e21cc10489f920bbbf3863d1475"

Inputs: ("Question: Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other? Choices: A: A)21, B: B)21.5, C: C)22, D: D)22.5, E: E)23", 'Question: In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively? Choices: A: A)4 and 1, B: B)1 and 5, C: C)5 and 1, D: D)3 and 5, E: E)5 and 3', 'Question: For all numbers p and q, the operation @ is defined by p@q = p^2 - pq. If xy ≠ 0, then which of the following can be equal to zero?\nI. x@y\nII. (xy)@y\nIII. x@(x + y) Choices: A: A)II, B: B)I and II, C: C)I and III, D: D)II and III, E: E)All of the above', 'Question: Carl is facing very difficult financial times and can only pay the interest on a $10,000 loan he has taken. The bank charges him a quarterly compound rate

In [4]:
def call_with_messages(model: str, content: str):
    messages = [{'role': 'user', 'content': content}]
    response = dashscope.Generation.call(
        model,    
        messages=messages,    
        result_format='message',   
        max_tokens=300
    )
    if response.status_code == HTTPStatus.OK:
        return response.output.choices[0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
    ))

def inference(input):
    outputs = call_with_messages('qwen-long', input)
    # print("outputs:", outputs)
    return outputs

trigger_sentences = [
    "Let's think step by step.",
    "We should think about this step by step.",
    "First,",
    "Before we dive into the answer,",
    "Proof followed by the answer.",
    "Let's think step by step in a realistic way.",
    "Let's think step by step using common sense and knowledge.",
    "Let's think like a detective step by step.",
    "Let's think about this logically.",
    "Let's think step by step. First,",
    "Let's think",
    "Let's solve this problem by splitting it into steps.",
    "The answer is after the proof.",
    "Let's be realistic and think step by step."
]


In [5]:
epochs = 1
num_paths = 3  # Number of reasoning paths to generate
accuracies = []


In [6]:
def most_common(lst):
    return max(set(lst), key=lst.count)

def sec_common(lst):
    if len(lst) < 2:
        return None  # Not enough elements for a second most common
    most_common_element = most_common(lst)
    # Remove all occurrences of the most common element
    filtered_lst = [elem for elem in lst if elem != most_common_element]
    if not filtered_lst:
        return None  # No other elements left for a second most common
    return most_common(filtered_lst)

In [7]:
# 定义验证函数
def verify_answer(question, cot):
    verification_prompt = f"Q: {question}. Is the inference process {cot} correct? Just reply me with yes or no"
    verification_result = call_with_messages('qwen-turbo', verification_prompt)
    print(verification_result)
    if verification_result:
        return "yes" in verification_result.lower()
    else:
        print("Error in getting verification output.")
        return False

# 定义重新推理函数
def re_inference(input):
    outputs = call_with_messages('qwen-turbo', input)
    if outputs:
        print("Re-inference outputs:", outputs)
        return outputs
    else:
        print("Error in getting re-inference output.")
        return ""


In [8]:
for epoch in range(epochs):
    correct_count = 0
    total_count = 0
    
    for batch in aqua_dataloader:  # 修改为aqua_dataloader
        inputs, labels = batch

        for question, true_label in zip(inputs, labels):
            path_answers = []
            for _ in range(num_paths):
                trigger_sentence = random.choice(trigger_sentences)
                print(f"selecting {trigger_sentence}")
                X0 = f"Q: {question} A: {trigger_sentence}"
                Z = inference(X0)
                final_prompt = f"{X0} {Z} Therefore, among A through E, the answer is(only one alphabet, no need to write the whole word, e.g., 'A' or 'B' or 'C' or 'D' or 'E')"
                final_answer = inference(final_prompt)
                prediction = None
                for char in final_answer:
                    if char.isupper():
                        prediction = char
                        break
                path_answers.append(prediction)
            print(path_answers)
            # Use self-consistency to select the most common answer
            final_answer = ""
            first_prediction = most_common(path_answers)
            second_prediction = sec_common(path_answers)
            if second_prediction != None:
                # 如果存在第二种答案，用verifier检验
                print("verifying answer...")
                cot = f"{Z} Therefore, among A through E, the answer is {first_prediction}"
                if not verify_answer(question, cot):
                    reinference_prompt = f"{X0}" + f" The answer is not {first_prediction}." + f" Therefore, the answer is(only one alphabet, no need to write the whole word, e.g., 'A' or 'B' or 'C' or 'D' or 'E') "
                    print("verify failed, reinferencing...")
                    corrected_answer = re_inference(reinference_prompt)
                    prediction = ""
                    for char in corrected_answer:
                        if char.isupper():
                            prediction = char
                            break
                    final_answer = prediction
                else:
                    final_answer = first_prediction
            else:
                final_answer = first_prediction
            print(f"final_answer: {final_answer}")
            if final_answer == true_label:
                correct_count += 1
            total_count += 1
        break
    accuracy = correct_count / total_count if total_count > 0 else 0
    accuracies.append(accuracy)
    print(f"Epoch {epoch + 1}/{epochs} - Accuracy: {accuracy * 100:.2f}%")

print("Accuracies over epochs:", accuracies)

selecting Let's think step by step.
selecting Let's be realistic and think step by step.
selecting Let's think step by step. First,
['D', 'D', 'D']
final_answer: D
selecting Let's think like a detective step by step.
selecting First,
selecting Let's think step by step. First,
['C', 'C', 'C']
final_answer: C
selecting Let's think step by step.
selecting Let's think step by step. First,
selecting Before we dive into the answer,
['C', 'E', 'C']
verifying answer...
Yes.
final_answer: C
selecting Proof followed by the answer.
selecting Let's solve this problem by splitting it into steps.
selecting Let's think step by step in a realistic way.
['C', 'C', 'C']
final_answer: C
selecting Let's solve this problem by splitting it into steps.
selecting Let's be realistic and think step by step.
selecting Let's think step by step using common sense and knowledge.
['E', 'E', 'E']
final_answer: E
selecting Let's think step by step.
selecting Let's think like a detective step by step.
selecting Let's b